In [5]:
import warnings
warnings.filterwarnings('ignore')
import os
import monai
import torch
import numpy as np
import pandas as pd
import tqdm as tqdm
from monai.data import decollate_batch
from torch.utils.data import DataLoader
from monai.metrics import ROCAUCMetric
from monai.utils import set_determinism
import matplotlib.pyplot as plt
import PIL
from monai.transforms import LoadImage, Resize, NormalizeIntensity,RandRotate,RandFlip,RandZoom,Compose,Activations, AsDiscrete,EnsureChannelFirst
from monai.networks.nets import DenseNet121
import torchvision.transforms as transforms
import pydicom
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report
import seaborn as sns
from sklearn.model_selection import train_test_split

In [6]:
set_determinism(seed = 1234)
data_dir  = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/dcm/test/"
image_files = [os.path.join(data_dir,x) for x in os.listdir(data_dir)]

In [11]:
val_transforms = Compose([LoadImage(image_only=True),EnsureChannelFirst() ,NormalizeIntensity(),Resize((256,256))])
y_pred_trans = Compose([Activations(softmax=True)])
y_trans = Compose([AsDiscrete(to_onehot=2)])

class BrainClassificationDatasetinference(torch.utils.data.Dataset):
    def __init__(self, image_files, transforms):
        self.image_files = image_files
        
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index])

    
test_ds = BrainClassificationDatasetinference(image_files,val_transforms)
test_loader = DataLoader(test_ds, batch_size=20,num_workers=5)

In [26]:
test_images = next(iter(test_loader))
print(f"Images batch shape: {test_images.size()}")


Images batch shape: (20, 1, 256, 256)


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=2).to(device)

In [41]:
name  = list()
prediction  = list ()
root_dir = "./model_weight/"
model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
    for test_data in test_loader:
        test_images = (test_data.to(device))
        pred = model(test_images).argmax(dim=1)
        for i in range(len(pred)):
            prediction.append(pred[i].item())
            name.append(pred[i].meta["filename_or_obj"])


In [51]:
predizioni = {
    "path" : name ,
    "predizione" : prediction
}

df  = pd.DataFrame.from_dict(predizioni).to_csv("./risultati/prova_super_test.csv")
